# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with read_sql_table
- Define feature and target variables X and Y

In [5]:
# import libraries
import sys
import pandas as pd
import numpy as np
from sqlalchemy import create_engine

from nltk.tokenize import word_tokenize,sent_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer

from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.multioutput import MultiOutputClassifier

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import precision_recall_fscore_support,accuracy_score
from sklearn.model_selection import GridSearchCV

import pickle

In [2]:
import nltk
nltk.download(['punkt', 'wordnet','stopwords'])

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Herkules\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Herkules\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Herkules\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql_table("DisasterResponseMessages", con=engine)
X = df['message']
Y = df.drop(['id','message','original','genre'],axis=1)

Y.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 2. Write a tokenization function to process your text data

In [4]:
def tokenize(text):
    """ 
    Function to transform text in tokens
    parameters:
        text: text to be transformed
    output:
        list of tokens
    """
    
    words = word_tokenize(text)
    tokens = [w for w in words if w not in stopwords.words("english")]
    
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [5]:
pipeline = Pipeline([
        ('vect', TfidfVectorizer(tokenizer=tokenize)),
        ('clf', MultiOutputClassifier(RandomForestClassifier(n_estimators= 50, random_state=42)))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [6]:
# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2, random_state = 42)

# Train pipeline
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('vect',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(...
                 MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True,
                                                                        class_weight=None,
                                                                        criterion='gini',
                                 

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [7]:
y_pred = pipeline.predict(X_test)

for i, col in enumerate(y_test):
        
        accuracy=accuracy_score(y_test.loc[:,col],y_pred[:,i])
        print("[{:}] - accuracy: {:.2f}\n".format(col,accuracy))
        print(classification_report(y_test[col], y_pred[:, i]))
        

[related] - accuracy: 0.80

              precision    recall  f1-score   support

           0       0.71      0.35      0.47      1266
           1       0.81      0.95      0.88      3938
           2       0.71      0.12      0.21        40

    accuracy                           0.80      5244
   macro avg       0.74      0.48      0.52      5244
weighted avg       0.79      0.80      0.77      5244

[request] - accuracy: 0.89

              precision    recall  f1-score   support

           0       0.90      0.98      0.94      4349
           1       0.84      0.47      0.61       895

    accuracy                           0.89      5244
   macro avg       0.87      0.73      0.77      5244
weighted avg       0.89      0.89      0.88      5244

[offer] - accuracy: 1.00

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      5218
           1       0.00      0.00      0.00        26

    accuracy                           1.00   

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


[death] - accuracy: 0.96

              precision    recall  f1-score   support

           0       0.96      1.00      0.98      5007
           1       0.77      0.16      0.26       237

    accuracy                           0.96      5244
   macro avg       0.87      0.58      0.62      5244
weighted avg       0.95      0.96      0.95      5244

[other_aid] - accuracy: 0.87

              precision    recall  f1-score   support

           0       0.87      1.00      0.93      4549
           1       0.55      0.03      0.06       695

    accuracy                           0.87      5244
   macro avg       0.71      0.51      0.50      5244
weighted avg       0.83      0.87      0.81      5244

[infrastructure_related] - accuracy: 0.94

              precision    recall  f1-score   support

           0       0.94      1.00      0.97      4916
           1       1.00      0.00      0.01       328

    accuracy                           0.94      5244
   macro avg       0.97      

### 6. Improve your model
Use grid search to find better parameters. 

In [8]:
parameters =  {'clf__estimator__n_estimators': [50,100] } 

cv = GridSearchCV(pipeline, param_grid=parameters, cv=3, n_jobs=1)

In [9]:
# Split data into train and test sets
#X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2, random_state = 42)

cv.fit(X_train, y_train)
y_pred = cv.predict(X_test)

In [10]:
cv.best_params_

{'clf__estimator__n_estimators': 100}

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [11]:
y_pred = cv.best_estimator_.predict(X_test)

for i, col in enumerate(y_test):
        
        accuracy=accuracy_score(y_test.loc[:,col],y_pred[:,i])
        print("[{:}] - accuracy: {:.2f}\n".format(col,accuracy))
        print(classification_report(y_test[col], y_pred[:, i]))

[related] - accuracy: 0.80

              precision    recall  f1-score   support

           0       0.72      0.34      0.46      1266
           1       0.81      0.96      0.88      3938
           2       0.83      0.12      0.22        40

    accuracy                           0.80      5244
   macro avg       0.79      0.47      0.52      5244
weighted avg       0.79      0.80      0.77      5244

[request] - accuracy: 0.89

              precision    recall  f1-score   support

           0       0.90      0.98      0.94      4349
           1       0.85      0.47      0.60       895

    accuracy                           0.89      5244
   macro avg       0.87      0.73      0.77      5244
weighted avg       0.89      0.89      0.88      5244

[offer] - accuracy: 1.00

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      5218
           1       0.00      0.00      0.00        26

    accuracy                           1.00   

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)



[clothing] - accuracy: 0.99

              precision    recall  f1-score   support

           0       0.99      1.00      0.99      5171
           1       1.00      0.07      0.13        73

    accuracy                           0.99      5244
   macro avg       0.99      0.53      0.56      5244
weighted avg       0.99      0.99      0.98      5244

[money] - accuracy: 0.98

              precision    recall  f1-score   support

           0       0.98      1.00      0.99      5140
           1       0.89      0.08      0.14       104

    accuracy                           0.98      5244
   macro avg       0.94      0.54      0.57      5244
weighted avg       0.98      0.98      0.97      5244

[missing_people] - accuracy: 0.99

              precision    recall  f1-score   support

           0       0.99      1.00      0.99      5184
           1       1.00      0.02      0.03        60

    accuracy                           0.99      5244
   macro avg       0.99      0.51    

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [14]:
# Another machine learning algorithms: AdaBoostClassifier
other_pipeline  = Pipeline([
        ('vect', TfidfVectorizer(tokenizer=tokenize)),
        ('clf', MultiOutputClassifier(AdaBoostClassifier(random_state=42)))
])

other_pipeline.fit(X_train, y_train)

y_pred = other_pipeline.predict(X_test)

for i, col in enumerate(y_test):
        
        accuracy=accuracy_score(y_test.loc[:,col],y_pred[:,i])
        print("[{:}] - accuracy: {:.2f}\n".format(col,accuracy))
        print(classification_report(y_test[col], y_pred[:, i]))

[related] - accuracy: 0.75

              precision    recall  f1-score   support

           0       0.50      0.08      0.14      1266
           1       0.76      0.97      0.86      3938
           2       0.50      0.20      0.29        40

    accuracy                           0.75      5244
   macro avg       0.59      0.42      0.43      5244
weighted avg       0.70      0.75      0.68      5244

[request] - accuracy: 0.89

              precision    recall  f1-score   support

           0       0.90      0.97      0.94      4349
           1       0.79      0.49      0.60       895

    accuracy                           0.89      5244
   macro avg       0.85      0.73      0.77      5244
weighted avg       0.88      0.89      0.88      5244

[offer] - accuracy: 0.99

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      5218
           1       0.14      0.04      0.06        26

    accuracy                           0.99   


[fire] - accuracy: 0.99

              precision    recall  f1-score   support

           0       0.99      1.00      1.00      5191
           1       0.59      0.25      0.35        53

    accuracy                           0.99      5244
   macro avg       0.79      0.62      0.67      5244
weighted avg       0.99      0.99      0.99      5244

[earthquake] - accuracy: 0.97

              precision    recall  f1-score   support

           0       0.98      0.99      0.98      4766
           1       0.87      0.78      0.82       478

    accuracy                           0.97      5244
   macro avg       0.92      0.89      0.90      5244
weighted avg       0.97      0.97      0.97      5244

[cold] - accuracy: 0.98

              precision    recall  f1-score   support

           0       0.98      1.00      0.99      5127
           1       0.74      0.29      0.42       117

    accuracy                           0.98      5244
   macro avg       0.86      0.64      0.70   

### 9. Export your model as a pickle file

In [15]:
with open('pipeline_model.pkl', 'wb') as f:
    pickle.dump(cv.best_estimator_, f)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.

In [8]:
# import libraries
import pandas as pd
import numpy as np
from sqlalchemy import create_engine

from nltk.tokenize import word_tokenize,sent_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer

from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.multioutput import MultiOutputClassifier

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import precision_recall_fscore_support,accuracy_score
from sklearn.model_selection import GridSearchCV

import pickle

import nltk
nltk.download(['punkt', 'wordnet','stopwords'])


def load_data(database_filepath):
    """
    Funtion to load data from database
    Parameter: 
        database_filepath: path of the database 
    Returns:
        - X: features (messages)
        - Y: Labels (categories)
        - categories_names: columns of the labels
    """ 
    engine = create_engine('sqlite:///{:}'.format(database_filepath))
    df = pd.read_sql_table("DisasterResponseMessages", con=engine)
    X = df['message']
    Y = df.drop(['id','message','original','genre'],axis=1)
    
    return X, Y, Y.columns

def tokenize(text):
    """ 
    Function to transform text in tokens
    parameters:
        text: text to be transformed
    Return:
        list of tokens
    """
    
    words = word_tokenize(text)
    tokens = [w for w in words if w not in stopwords.words("english")]
    
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens


def build_model():
    """
    Function to build a model for machine learning
    return:
        model for machine learning
    """
    pipeline = Pipeline([
        ('vect', TfidfVectorizer(tokenizer=tokenize)),
        ('clf', MultiOutputClassifier(RandomForestClassifier(n_estimators= 50, random_state=42)))
    ])
    
    return pipeline

def evaluate_model(model, X_test, Y_test, category_names):
    """
    Function to evaluate a model
    Parameters:
        model: The model to be evaluated
        X_test: features for testing
        Y_test: labels for testing
        category_names: columns of the labels
    """
    
    Y_pred = model.predict(X_test)

    for i, col in enumerate(category_names):

            accuracy=accuracy_score(Y_test.loc[:,col],Y_pred[:,i])
            print("[{:}] - accuracy: {:.2f}\n".format(col,accuracy))
            print(classification_report(Y_test[col], Y_pred[:, i]))


def save_model(model, model_filepath):
    """
    Function to save a model
    Parameters:
        model: model to be saved
        model_filepath: file path where the model will be saved
    """
    
    with open('model_filepath', 'wb') as f:
        pickle.dump(model, f)


def main():
    if len(sys.argv) == 3:
        database_filepath, model_filepath = sys.argv[1:]
        print('Loading data...\n    DATABASE: {}'.format(database_filepath))
        X, Y, category_names = load_data(database_filepath)
        X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
        
        print('Building model...')
        model = build_model()
        
        print('Training model...')
        model.fit(X_train, Y_train)
        
        print('Evaluating model...')
        evaluate_model(model, X_test, Y_test, category_names)

        print('Saving model...\n    MODEL: {}'.format(model_filepath))
        save_model(model, model_filepath)

        print('Trained model saved!')

    else:
        print('Please provide the filepath of the disaster messages database '\
              'as the first argument and the filepath of the pickle file to '\
              'save the model to as the second argument. \n\nExample: python '\
              'train_classifier.py ../data/DisasterResponse.db classifier.pkl')


#if __name__ == '__main__':
#    print("main main...")
#    main()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
main main...
Please provide the filepath of the disaster messages database as the first argument and the filepath of the pickle file to save the model to as the second argument. 

Example: python train_classifier.py ../data/DisasterResponse.db classifier.pkl
